In [268]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [270]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [273]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially.

         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`.
    """

    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})


        Just write a little loop.
        """

        # Your code goes here. ################################################
        self.output = input
        for module in self.modules:
            self.output = module.forward(self.output)
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)


        !!!

        To ech module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!

        """
        # Your code goes here. ################################################
        self.gradInput = gradOutput
        for i in range(len(self.modules)-1, -1, -1):
            module = self.modules[i]
            if i > 0:
                prev_output = self.modules[i-1].output
            else:
                prev_output = input
            self.gradInput = module.backward(prev_output, self.gradInput)
        return self.gradInput


    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [276]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.dot(input, self.W.T) + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if gradOutput is None:
            self.gradInput = np.zeros_like(input)
            return self.gradInput
            
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW += np.dot(gradOutput.T, input)
        self.gradb += np.sum(gradOutput, axis=0)
        pass

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [278]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))

        # Your code goes here. ################################################
        self.output = np.divide(np.exp(self.output), np.sum(np.exp(self.output), axis=1, keepdims=True))
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size, num_class = self.output.shape
        self.gradInput = np.zeros_like(gradOutput)
        
        for i in range(batch_size):
            s = self.output[i].reshape(-1, 1)
            jacob = np.diagflat(s) - np.dot(s, s.T)
            self.gradInput[i] = np.dot(jacob, gradOutput[i])

        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [341]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))

        # Your code goes here. ################################################
        exp_shifted = np.exp(self.output)
        sum_exp = np.sum(exp_shifted, axis=1, keepdims=True)
        
        self.output = self.output - np.log(sum_exp)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        softmax = np.exp(self.output)
        if self.gradInput is None or self.gradInput.shape != input.shape:
            self.gradInput = np.zeros_like(input)
        sum_grad = np.sum(gradOutput, axis=1, keepdims=True)
        self.gradInput = gradOutput - softmax * sum_grad
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [282]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        if self.training:
            batch_mean = np.mean(input, axis=0)
            batch_var = np.var(input, axis=0)
            
            self.batch_mean = batch_mean
            self.batch_var = batch_var
            self.output = (input - batch_mean) / np.sqrt(batch_var + self.EPS)
            
            if self.moving_mean is None:
                self.moving_mean = batch_mean.copy()
                self.moving_variance = batch_var.copy()
            else:
                self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * batch_mean
                self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * batch_var
        else:
            self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.training:
            N = input.shape[0]
            std_inv = 1.0 / np.sqrt(self.batch_var + self.EPS)
            xhat = (input - self.batch_mean) * std_inv
            dxhat = gradOutput
            dvar = np.sum(dxhat * (input - self.batch_mean) * (-0.5) * std_inv**3, axis=0)
            dmean = np.sum(-dxhat * std_inv, axis=0) + dvar * np.mean(-2.0 * (input - self.batch_mean), axis=0)
            self.gradInput = dxhat * std_inv + dvar * 2.0 * (input - self.batch_mean) / N + dmean / N
        else:
            std_inv = 1.0 / np.sqrt(self.moving_variance + self.EPS)
            self.gradInput = gradOutput * std_inv
        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [283]:
class ChannelwiseScaling(Module):
    r"""
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [286]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.training:
            self.mask = np.random.binomial(1, 1 - self.p, size=input.shape) / (1 - self.p)
            self.output = input * self.mask
        else:
            self.output = input
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.training:
            self.gradInput = gradOutput * self.mask
        else:
            self.gradInput = gradOutput 
        return self.gradInput

    def __repr__(self):
        return "Dropout"

# 6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [288]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size, 
                 stride=1, padding=0, bias=True, padding_mode='zeros'):
        super().__init__()
        
        # Параметры свертки
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)
        self.padding_mode = padding_mode
        
        # Инициализация весов
        stdv = 1. / np.sqrt(in_channels * np.prod(self.kernel_size))
        self.weights = np.random.uniform(-stdv, stdv, 
                                        size=(out_channels, in_channels, *self.kernel_size))
        self.bias = np.random.uniform(-stdv, stdv, size=out_channels) if bias else None
        
        # Градиенты
        self.grad_weights = np.zeros_like(self.weights)
        self.grad_bias = np.zeros_like(self.bias) if bias else None
        
    def _pad_input(self, input):
        """Добавляет padding к входному тензору"""
        if self.padding[0] == 0 and self.padding[1] == 0:
            return input
            
        return np.pad(input, 
                     ((0, 0), (0, 0), 
                      (self.padding[0], self.padding[0]), 
                      (self.padding[1], self.padding[1])),
                     mode=self.padding_mode)
    
    def updateOutput(self, input):
        batch_size, in_channels, in_h, in_w = input.shape
        
        if in_channels != self.in_channels:
            raise ValueError(f"Expected input with {self.in_channels} channels, got {in_channels}")
            
        padded_input = self._pad_input(input)
        
        out_h = (in_h + 2*self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1
        out_w = (in_w + 2*self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1
        
        self.output = np.zeros((batch_size, self.out_channels, out_h, out_w))
        
        for b in range(batch_size):
            for c_out in range(self.out_channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride[0]
                        w_start = w * self.stride[1]
                        receptive_field = padded_input[b, :, 
                                                     h_start:h_start+self.kernel_size[0], 
                                                     w_start:w_start+self.kernel_size[1]]
                        
                        self.output[b, c_out, h, w] = np.sum(
                            receptive_field * self.weights[c_out]
                        )
                
                if self.bias is not None:
                    self.output[b, c_out] += self.bias[c_out]
        
        self.input = input  # сохраняем для backward
        return self.output
    
    def updateGradInput(self, input, grad_output):
        batch_size, _, out_h, out_w = grad_output.shape
        
        self.gradInput = np.zeros_like(input)
        
        padded_grad = np.zeros((batch_size, self.in_channels,
                              input.shape[2] + 2*self.padding[0],
                              input.shape[3] + 2*self.padding[1]))
        
        for b in range(batch_size):
            for c_out in range(self.out_channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride[0]
                        w_start = w * self.stride[1]
                        
                        padded_grad[b, :, 
                                   h_start:h_start+self.kernel_size[0],
                                   w_start:w_start+self.kernel_size[1]] += \
                            self.weights[c_out] * grad_output[b, c_out, h, w]
        
        if self.padding[0] > 0 or self.padding[1] > 0:
            self.gradInput = padded_grad[:, :, 
                                       self.padding[0]:-self.padding[0],
                                       self.padding[1]:-self.padding[1]]
        else:
            self.gradInput = padded_grad
            
        return self.gradInput
    
    def accGradParameters(self, input, grad_output):
        batch_size, _, out_h, out_w = grad_output.shape
        
        for b in range(batch_size):
            for c_out in range(self.out_channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride[0]
                        w_start = w * self.stride[1]
                        receptive_field = self._pad_input(input)[b, :, 
                                                               h_start:h_start+self.kernel_size[0],
                                                               w_start:w_start+self.kernel_size[1]]
                        
                        self.grad_weights[c_out] += receptive_field * grad_output[b, c_out, h, w]
        
        if self.bias is not None:
            self.grad_bias += np.sum(grad_output, axis=(0, 2, 3))
    
    def zeroGradParameters(self):
        self.grad_weights.fill(0)
        if self.bias is not None:
            self.grad_bias.fill(0)
    
    def getParameters(self):
        return [self.weights, self.bias] if self.bias is not None else [self.weights]
    
    def getGradParameters(self):
        return [self.grad_weights, self.grad_bias] if self.bias is not None else [self.grad_weights]
    
    def __repr__(self):
        return "Conv2d"

#7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [290]:
class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def updateOutput(self, input):
        # Your code goes here. ################################################
        batch_size, in_channels, in_h, in_w = input.shape

        out_h = (in_h + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_w = (in_w + 2 * self.padding - self.kernel_size) // self.stride + 1
        self.output = np.zeros((batch_size, in_channels, out_h, out_w))

        if self.padding > 0:
            input_padded = np.pad(input, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode="constant")
        else:
            input_padded = input
        self.max_indices = np.zeros_like(input_padded)  #сохраняем индексы максимальных значений для backward pass
    
        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start, w_start = i * self.stride, j * self.stride
                        h_end, w_end = h_start + self.kernel_size, w_start + self.kernel_size
                        area = input_padded[b, c, h_start:h_end, w_start:w_end]
                        
                        max_value = np.max(area)
                        self.output[b, c, i, j] = max_value

                        max_mask = (area == max_value)
                        self.max_indices[b, c, h_start:h_end, w_start:w_end] += max_mask
    
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size, in_channels, in_h, in_w = input.shape
        _, _, out_h, out_w = gradOutput.shape

        self.gradInput = np.zeros_like(input)
    
        if self.padding > 0:
            gradInput_padded = np.pad(self.gradInput, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode="constant")
        else:
            gradInput_padded = self.gradInput
    
        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start, w_start = i * self.stride, j * self.stride
                        h_end, w_end = h_start + self.kernel_size, w_start + self.kernel_size
    
                        gradInput_padded[b, c, h_start:h_end, w_start:w_end] += self.max_indices[b, c, h_start:h_end, w_start:w_end] * gradOutput[b, c, i, j]
        if self.padding > 0:
            self.gradInput = gradInput_padded[:, :, self.padding:-self.padding, self.padding:-self.padding]
        else:
            self.gradInput = gradInput_padded
        return self.gradInput

    def __repr__(self):
        return "MaxPool2d"

class AvgPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(AvgPool2d, self).__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def updateOutput(self, input):
        # Your code goes here. ################################################
        batch_size, in_channels, in_h, in_w = input.shape
 
        out_h = (in_h + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_w = (in_w + 2 * self.padding - self.kernel_size) // self.stride + 1
        self.output = np.zeros((batch_size, in_channels, out_h, out_w))
    
        if self.padding > 0:
            input_padded = np.pad(input, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode="constant")
        else:
            input_padded = input
    
        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start, w_start = i * self.stride, j * self.stride
                        h_end, w_end = h_start + self.kernel_size, w_start + self.kernel_size
                        area = input_padded[b, c, h_start:h_end, w_start:w_end]
    
                        self.output[b, c, i, j] = np.mean(area)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size, in_channels, in_h, in_w = input.shape
        _, _, out_h, out_w = gradOutput.shape

        self.gradInput = np.zeros_like(input)
    
        if self.padding > 0:
            gradInput_padded = np.pad(self.gradInput, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode="constant")
        else:
            gradInput_padded = self.gradInput
    
        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start, w_start = i * self.stride, j * self.stride
                        h_end, w_end = h_start + self.kernel_size, w_start + self.kernel_size
    
                        grad_value = gradOutput[b, c, i, j] / (self.kernel_size ** 2)
                        gradInput_padded[b, c, h_start:h_end, w_start:w_end] += grad_value
    
        if self.padding > 0:
            self.gradInput = gradInput_padded[:, :, self.padding:-self.padding, self.padding:-self.padding]
        else:
            self.gradInput = gradInput_padded
        return self.gradInput

    def __repr__(self):
        return "AvgPool2d"

#8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [292]:
class GlobalMaxPool2d(Module):
    def __init__(self):
        super(GlobalMaxPool2d, self).__init__()

    def updateOutput(self, input):
        batch_size, in_channels, in_h, in_w = input.shape
        self.output = np.max(input, axis=(2, 3))  #max по высоте и ширине
        return self.output

    def updateGradInput(self, input, gradOutput):
        batch_size, in_channels, in_h, in_w = input.shape
        _, _, out_h, out_w = gradOutput.shape

        self.gradInput = np.zeros_like(input)
        max_indices = (input == np.expand_dims(self.output, axis=(2, 3))) 
        self.gradInput = max_indices * gradOutput[:, :, :, np.newaxis, np.newaxis]  

        return self.gradInput

    def __repr__(self):
        return "GlobalMaxPool2d"


In [293]:
class GlobalAvgPool2d(Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def updateOutput(self, input):
        batch_size, in_channels, in_h, in_w = input.shape
        self.output = np.mean(input, axis=(2, 3))  # mean по высоте и ширине
        return self.output

    def updateGradInput(self, input, gradOutput):
        batch_size, in_channels, in_h, in_w = input.shape
        _, _, out_h, out_w = gradOutput.shape

        self.gradInput = np.ones_like(input) * gradOutput[:, :, :, np.newaxis, np.newaxis]  
        self.gradInput /= (in_h * in_w)  

        return self.gradInput

    def __repr__(self):
        return "GlobalAvgPool2d"


#9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [295]:
class Flatten(Module):
    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()

        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.input_shape = input.shape  

        start_dim = self.start_dim if self.start_dim >= 0 else len(input.shape) + self.start_dim
        end_dim = self.end_dim if self.end_dim >= 0 else len(input.shape) + self.end_dim

        new_shape = (input.shape[:start_dim] + (-1,) + input.shape[end_dim + 1:])
        self.output = input.reshape(new_shape)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [298]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [300]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()

        self.slope = slope

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.where(input > 0, input, self.slope * input)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.where(input > 0, 1, self.slope)
        self.gradInput *= gradOutput  
        return self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [302]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.where(input > 0, input, self.alpha * (np.exp(input) - 1))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.where(input > 0, 1, self.alpha * np.exp(input))
        self.gradInput *= gradOutput
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [304]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.log(1 + np.exp(input))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = 1 / (1 + np.exp(-input))
        self.gradInput *= gradOutput
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

#13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [306]:
class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        c = np.sqrt(2 / np.pi)
        self.output = 0.5 * input * (1 + np.tanh(c * (input + 0.044715 * np.power(input, 3))))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        c = np.sqrt(2 / np.pi)
        tan = np.tanh(c * (input + 0.044715 * input**3))
        sec = 1 - tan**2 
        gelu_grad = 0.5 * (1 + tan) + 0.5 * input * sec * c * (1 + 3 * 0.044715 * np.power(input, 2))
        self.gradInput = gradOutput * gelu_grad
        return self.gradInput

    def __repr__(self):
        return "Gelu"

# Criterions

Criterions are used to score the models answers.

In [309]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [311]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [313]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        # Your code goes here. ################################################
        batch_size = input.shape[0]
        self.output = -np.sum(target * np.log(input_clamp)) / batch_size
        return self.output

    def updateGradInput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        # Your code goes here. ################################################
        batch_size = input.shape[0]
        
        # Создаем новый массив нужного размера вместо использования 'out='
        self.gradInput = -target / (input_clamp * batch_size)
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [315]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        # a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        # Your code goes here. ################################################
        batch_size = input.shape[0]
        self.output = -np.sum(target * input) / batch_size 
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        batch_size = input.shape[0]
        self.gradInput = -target / batch_size 
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вашими классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.